# **Cointab Software Private Limited**

# *Data Analyst - Assignment*

1. Include libraries and access or open documents.
2. File Paths
3. Read Files
4. Null Values
5. Duplicate Values
6. Drop Duplicates
7. Merge
8. Total Weight in KG
9. Drop Unnecessary Columns
10. Total Weight and Amount for Each Order
11. Merge with Invoice Data
12. Merge with Pincode Data
13. Convert Uppercase to Lowercase
14. Forward Additional Charge
15. Fixed and Additional RTO Charges
16. Calculate Total Expected Charge for Delivery
17. Difference between Expected and Billed Charges
18. Add Column for Weight Slabs Charged by Courier for Output
19. Add Column to Check if Courier Company Overcharged or Undercharged
20. Output
21. Summary
22. Save to Excel
23. output.to_excel("/content/Output.xlsx")
24. summary.to_excel("/content/Summary.xlsx")


In [7]:

import math
import numpy as np
import pandas as pd




files = ["/content/Company X - Order Report.xlsx", "/content/Company X - Pincode Zones.xlsx",
         "/content/Company X - SKU Master.xlsx", "/content/Courier Company - Invoice.xlsx",
         "/content/Courier Company - Rates.xlsx"]




order_report, pin_zone, sku, invoice, rates = [pd.read_excel(file) for file in files]






missing_values = {f"{file}": df.isna().sum() for file, df in zip(files, [order_report, pin_zone, sku, invoice, rates])}
print("\n".join([f"Missing Values in {file}:\n{values}" for file, values in missing_values.items()]))





duplicate_values = {f"{file}": df.duplicated().sum() for file, df in zip(files, [order_report, pin_zone, sku, invoice, rates])}
print("\n".join([f"Number of duplicate rows in {file}: {value}" for file, value in duplicate_values.items()]))





order_report, sku, pin_zone = [df.drop_duplicates() for df in [order_report, sku, pin_zone]]



data = order_report.merge(sku, how='left', on='SKU')



data['Total_Weight_by_X(kg)'] = data['Weight (g)'] * data['Order Qty'] / 1000



data = data.drop(labels=['Order Qty', 'Weight (g)', 'SKU'], axis=1)



data = data.groupby(by=['ExternOrderNo'], as_index=False).sum()



data = data.merge(invoice, how='left', left_on='ExternOrderNo', right_on='Order ID')



data = data.merge(pin_zone, how='left', on=['Warehouse Pincode', 'Customer Pincode']).rename(columns={'Zone_x': 'Zone_by_Courier', 'Zone_y': 'Zone_by_X'})



rates['Zone'] = rates['Zone'].str.lower()
data = data.merge(rates, how='left', left_on='Zone_by_X', right_on='Zone')



data['Forward_additional_charge'] = (data['Total_Weight_by_X(kg)'] / data['Weight Slabs']).astype(int) * data['Forward Additional Weight Slab Charge']



data['RTO_fixed_charge'] = data.apply(lambda row: row['RTO Fixed Charge'] if row['Type of Shipment'] == 'Forward and RTO charges' else 0, axis=1)
data['RTO_additional_charge'] = data.apply(lambda row: math.floor(row['Total_Weight_by_X(kg)'] / row['Weight Slabs']) * row['RTO Additional Weight Slab Charge'] if row['Type of Shipment'] == 'Forward and RTO charges' else 0, axis=1)



data['Expected_charges'] = data[['Forward Fixed Charge', 'Forward_additional_charge', 'RTO_fixed_charge', 'RTO_additional_charge']].sum(axis=1)



data['diff_charges'] = data['Expected_charges'] - data['Billing Amount (Rs.)']



data['Weight_slabs_Courier'] = data['Zone_by_Courier'].map({'a': 0.25, 'b': 0.5, 'c': 0.75, 'd': 1.25, 'e': 1.5})



data['over_or_under_charged'] = np.select([data['diff_charges'] == 0, data['diff_charges'] > 0, data['diff_charges'] < 0],
                                           ['X correctly charged', 'X Undercharged', 'X Overcharged'])



output = data[['Order ID', 'AWB Code', 'Total_Weight_by_X(kg)', 'Weight Slabs', 'Charged Weight', 'Weight_slabs_Courier', 'Zone_by_X',
               'Zone_by_Courier', 'Expected_charges', 'Billing Amount (Rs.)', 'diff_charges']]




summary = pd.DataFrame(index=['Total orders where X has been correctly charged',
                              'Total Orders where X has been overcharged',
                              'Total Orders where X has been undercharged'], columns=['Count', 'Amount(Rs.)'])
summary['Count'] = [(data['diff_charges'] == 0).sum(),
                    (data['diff_charges'] < 0).sum(),
                    (data['diff_charges'] > 0).sum()]
summary['Amount(Rs.)'] = [data.loc[data['diff_charges'] == 0, 'Billing Amount (Rs.)'].sum(),
                          abs(data.loc[data['diff_charges'] < 0, 'diff_charges'].sum()),
                          data.loc[data['diff_charges'] > 0, 'diff_charges'].sum()]




output.to_excel("/content/Output.xlsx")
summary.to_excel("/content/Summary.xlsx")


Missing Values in /content/Company X - Order Report.xlsx:
ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64
Missing Values in /content/Company X - Pincode Zones.xlsx:
Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64
Missing Values in /content/Company X - SKU Master.xlsx:
SKU           0
Weight (g)    0
dtype: int64
Missing Values in /content/Courier Company - Invoice.xlsx:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64
Missing Values in /content/Courier Company - Rates.xlsx:
Zone                                     0
Weight Slabs                             0
Forward Fixed Charge                     0
Forward Additional Weight Slab Charge    0
RTO Fixed Charge                         0
RTO Additional Weight Slab Charge        0
dtype: int64
Number of duplicate ro